In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator
import nltk
import re
from nltk.corpus import stopwords
import string

In [2]:
!pip install wordcloud

In [3]:
data = pd.read_csv('filename.csv')
data.head()

,id,conversation_id,created_at,date,time,timezone,user_id,username,name,place,...,geo,source,user_rt_id,user_rt,retweet_id,reply_to,retweet_date,translate,trans_src,trans_dest
0,1598483154855395328,1598407650110148608,2022-12-02 01:04:17 UTC,2022-12-02,01:04:17,0,1357346674646024194,gryffoyn,Gryffoyn ✡️,NaN,...,NaN,NaN,NaN,NaN,NaN,"[{'screen_name': 'charliekirk11', 'name': 'Cha...",NaN,NaN,NaN,NaN
1,1598483154184331266,1598483154184331266,2022-12-02 01:04:16 UTC,2022-12-02,01:04:16,0,119927179,davidwferrell,David Ferrell,NaN,...,NaN,NaN,NaN,NaN,NaN,[],NaN,NaN,NaN,NaN
2,1598483147628638208,1598152584438902784,2022-12-02 01:04:15 UTC,2022-12-02,01:04:15,0,1160951919248117761,ken_braza,Ken Braza,NaN,...,NaN,NaN,NaN,NaN,NaN,"[{'screen_name': 'leezeldin', 'name': 'Lee Zel...",NaN,NaN,NaN,NaN
3,1598483122131460096,1598017144180346881,2022-12-02 01:04:09 UTC,2022-12-02,01:04:09,0,1520858324971167744,adrian86749524,Adrian,NaN,...,NaN,NaN,NaN,NaN,NaN,"[{'screen_name': 'connor17383', 'name': 'conno...",NaN,NaN,NaN,NaN
4,1598483117169311745,1598423123019563009,2022-12-02 01:04:08 UTC,2022-12-02,01:04:08,0,796430551200702464,_whyzguy_,🖤ᴡᴴʏᶻɢᵁʏ🖤,NaN,...,NaN,NaN,NaN,NaN,NaN,"[{'screen_name': 'el_sickario', 'name': 'Prest...",NaN,NaN,NaN,NaN


In [4]:
data.columns

Index(['id', 'conversation_id', 'created_at', 'date', 'time', 'timezone',
       'user_id', 'username', 'name', 'place', 'tweet', 'language', 'mentions',
       'urls', 'photos', 'replies_count', 'retweets_count', 'likes_count',
       'hashtags', 'cashtags', 'link', 'retweet', 'quote_url', 'video',
       'thumbnail', 'near', 'geo', 'source', 'user_rt_id', 'user_rt',
       'retweet_id', 'reply_to', 'retweet_date', 'translate', 'trans_src',
       'trans_dest'],
      dtype='object')

In [5]:
data.shape

(10005, 36)

In [6]:
data.dtypes

id                   int64
conversation_id      int64
created_at          object
date                object
time                object
timezone             int64
user_id              int64
username            object
name                object
place               object
tweet               object
language            object
mentions            object
urls                object
photos              object
replies_count        int64
retweets_count       int64
likes_count          int64
hashtags            object
cashtags            object
link                object
retweet               bool
quote_url           object
video                int64
thumbnail           object
near               float64
geo                float64
source             float64
user_rt_id         float64
user_rt            float64
retweet_id         float64
reply_to            object
retweet_date       float64
translate          float64
trans_src          float64
trans_dest         float64
dtype: object

In [7]:
data.isna().sum()

id                     0
conversation_id        0
created_at             0
date                   0
time                   0
timezone               0
user_id                0
username               0
name                   0
place               9998
tweet                  0
language               0
mentions               0
urls                   0
photos                 0
replies_count          0
retweets_count         0
likes_count            0
hashtags               0
cashtags               0
link                   0
retweet                0
quote_url           9271
video                  0
thumbnail           8710
near               10005
geo                10005
source             10005
user_rt_id         10005
user_rt            10005
retweet_id         10005
reply_to               0
retweet_date       10005
translate          10005
trans_src          10005
trans_dest         10005
dtype: int64

In [8]:
data=data[["username", "tweet", "language"]]
data

,username,tweet,language
0,gryffoyn,@charliekirk11 THE NEW RUSSIA?,en
1,davidwferrell,The @January6thCmte has done a wonderful job s...,en
2,ken_braza,@leezeldin Like they say in Russia...it's not ...,en
3,adrian86749524,@connor17383 @GaworMarkus @ZelenskyyUa Aren't ...,en
4,_whyzguy_,@el_sickario @brianhugstrees @MarkHamill who's...,en
...,...,...,...
10000,lmamtschak,@KyivIndependent Good news! Russia should not ...,en
10001,crazyheadimp,@Stako_tennis @PTTennis1 Why is it not relevan...,en
10002,tom17121437,@Gerashchenko_en I always knew this beautiful ...,en
10003,marxistaportuga,@Culter2000 @AntonDrago71 @calateofacho @Alter...,pt


In [9]:
data.isna().sum()

username    0
tweet       0
language    0
dtype: int64

In [10]:
data['language'].value_counts()

en     8493
pt      434
qme     246
it      231
und     121
ru       93
ja       71
in       58
es       51
fr       24
ca       17
fi       15
de       14
qht      13
th       12
ar       12
nl        9
ro        9
tl        9
vi        9
pl        8
zh        8
tr        5
hi        4
sv        4
cs        4
da        4
qst       3
zxx       3
am        3
et        3
no        3
fa        2
bn        2
ko        2
is        1
lv        1
uk        1
sr        1
ht        1
hu        1
Name: language, dtype: int64

In [11]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [12]:
stemmer = nltk.SnowballStemmer("english")

In [13]:
stopword=set(stopwords.words('english'))

In [14]:
def clean(text):
    text = str(text).lower()
    text = re.sub('\[.*?\]', '', text)
    text = re.sub('https?://\S+|www\.\S+', '', text)
    text = re.sub('<.*?>+', '', text)
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub('\n', '', text)
    text = re.sub('\w*\d\w*', '', text)
    text = [word for word in text.split(' ') if word not in stopword]
    text=" ".join(text)
    text = [stemmer.stem(word) for word in text.split(' ')]
    text=" ".join(text)
    return text

In [15]:
data["tweet"] = data["tweet"].apply(clean)

In [16]:
data["tweet"]

0                                               new russia
1         done wonder job far  must recommend stiffest ...
2        leezeldin like say russiait vote matter count ...
3         gawormarkus zelenskyyua arent ukrain amp alli...
4        elsickario brianhugstre markhamil whos gonna t...
                               ...                        
10000    kyivindepend good news russia evacu militari u...
10001    stakotenni  relev today live kill merciless ar...
10002    gerashchenkoen alway knew beauti song treasur ...
10003      calateofacho alterrega isabecarv uma narrati...
10004    embassyofrussia mfarussia pmsimferopol rusembu...
Name: tweet, Length: 10005, dtype: object